In [ ]:
#include <iostream>
#include <thread>
#include <vector>
#include <iostream>
#include <atomic>
#include "compapi.hpp"

const int N = 100;
// S(N) = 1 + 2*2 + 3*3 + 4*4 + .. + (N-1)*(N-1) + N*N
{
    int S = 0;
    for(int n = 1; n <= N; n++) S = S + n*n;
    cout << "S(" << N << ") = " << S << endl;
}

In [ ]:
// 1 //
{
    atomic_int ID = 0;
    int num_threads = 10;
    
    EventLog log;
    vector<thread> threads(num_threads);
    
    for(auto& t:threads)t=thread([&]{
        
    //////////////// inside a process ////////////////
        
        struct SumSquareWorker: public ComputeWorker{
            SumSquareWorker(unsigned pid, EventLog&log, int&_S):ComputeWorker(pid,log),S(_S){}
            bool on_read(unsigned ord,unsigned tag,unsigned pid,const BLOB&blob)override{
                return false;
            }
            bool on_write(unsigned&tag,BLOB&blob)override{
                return false;
            }
            int& S;
        };

        cout << "hello" << endl;

        int S = 0;
        SumSquareWorker worker(ID++,log,S);
        //worker.run();
        if(ID==0) cout << "S(" << N << ") = " << S << endl;
        
    //////////////////////////////////////////////////

    }); 
    for(auto& t:threads) t.join();
}

In [ ]:
// 2 //
{
    atomic_int ID = 0;
    int num_threads = 10;
    
    EventLog log;
    vector<thread> threads(num_threads);
    
    for(auto& t:threads)t=thread([&]{
        
    //////////////// inside a process ////////////////
        
        struct SumSquareTask: public Task{
            SumSquareTask(TaskEngine&te, int _n, int& _S):Task(te),n(_n),S(_S){}
            void on_run()override{ n = n*n; }
            void on_continue()override{ S = S + n; }
            void on_save(ostream&out)override{ out << n; }
            void on_load(istream&in)override{ in >> n; }
            int& S; int  n;
        };
        
        cout << "hello" << endl;
        TaskEngine eng(ID++,log);
        
        int S = 0;
        vector<SumSquareTask*> tasks(N);
        
        for(int n=1;n<=N;n++){
            tasks[n-1] = new SumSquareTask(eng,n,S);
            tasks[n-1]->submit();
        }

        //eng.run();
        
        if(ID==0) cout << "S(" << N << ") = " << S << endl;
        for(auto& t:tasks) delete t;
        
    //////////////////////////////////////////////////
        
    });
    for(auto& t:threads) t.join();
}

In [ ]:
// 3
{
    atomic_int ID = 0;
    int num_threads = 10;
    
    EventLog log;
    vector<thread> threads(num_threads);
    
    for(auto& t:threads)t=thread([&]{
        
    //////////////// inside a process ////////////////   
        
        struct NSquareTask{ int  n; };
        
        struct SumSquareMV: public MasterWorkers<NSquareTask>{
            SumSquareMV(unsigned num_workers,unsigned pid,EventLog& log, int&_S):
                MasterWorkers(num_workers,pid,log),S(_S),n(1){}
        
            bool on_get(NSquareTask&t)override{if(n>N)return false; t.n = n; n++; return true; }
            void on_run(NSquareTask&t)override{ t.n = t.n * t.n; }
            void on_put(NSquareTask&t)override{ S = S + t.n; }
            void on_save(const NSquareTask&t,ostream&out)override{ out << t.n; }
            void on_load(NSquareTask&t,istream&in)override{ in >> t.n; }
            int& S; int n;
        };
        
        cout << "hello" << endl;
        int num_workers = 5;
        
        int S = 0;
        SumSquareMV master_worker(num_workers,ID++,log,S);
        //master_worker.run();
        if(ID==0) cout << "S(" << N << ") = " << S << endl;
    
    //////////////////////////////////////////////////
    
    });
    for(auto& t:threads) t.join();
}